### Tensorflow Case
---
一个小例子展示tensorflow工作原理

In [1]:
# 使用 NumPy 生成假数据(phony data), 总共 100 个点.
x_data = np.float32(np.random.rand(2, 100)) # 随机输入,(2, 100)形状的矩阵
y_data = np.dot([0.100, 0.200], x_data) + 0.300 # 结果形状，（1， 100）

# 构造一个线性模型
b = tf.Variable(tf.zeros([1]))    # shape是一个长度是1的向量
W = tf.Variable(tf.random_uniform([1, 2], -1.0, 1.0))    # -1 ～ 1 之间的随机张量，形状是(1, 2)
y = tf.matmul(W, x_data) + b    # 矩阵乘法，开始拟合判断

# 最小化 **方差**
loss = tf.reduce_mean(tf.square(y - y_data))    # 定义方差计算节点
optimizer = tf.train.GradientDescentOptimizer(0.5)    # 定义优化器
train = optimizer.minimize(loss)    # 定义优化器最小化计算节点

# 初始化变量
init = tf.global_variables_initializer()

# 启动图 (graph)
sess = tf.Session()
sess.run(init)    # Variable节点初始化

# 拟合平面
for step in range(0, 201):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(W), sess.run(b))     # 输出会话器Session跟踪的值

# 得到最佳拟合结果 W: [[0.100  0.200]], b: [0.300]

0 [[ 0.8833946  -0.06681323]] [ 0.06528893]
20 [[ 0.28255695  0.14989944]] [ 0.23387896]
40 [[ 0.14345604  0.19283505]] [ 0.28187495]
60 [[ 0.11050617  0.19981292]] [ 0.29484379]
80 [[ 0.10259257  0.20044759]] [ 0.29848024]
100 [[ 0.10065655  0.20026791]] [ 0.29953769]
120 [[ 0.10017154  0.20011717]] [ 0.29985559]
140 [[ 0.10004642  0.20004565]] [ 0.29995394]
160 [[ 0.10001304  0.20001683]] [ 0.29998505]
180 [[ 0.10000382  0.20000602]] [ 0.29999509]
200 [[ 0.10000114  0.2000021 ]] [ 0.29999837]


### Tensorflow 基本使用说明
---

1. 工作模式
    * 使用计算图表示任务
        
        1. 图中存在节点，节点表示对张量Tensor的处理，输入多个(or 0)个张量，输出多个(or 0)个张量
        
    * 使用会话执行图
    
        1. 会话启动图
        2. 会话将图中要计算的节点分发到不同的 CPU / GPU 等计算设备中
    
    * Tensor传递数据 ： 多维数组的超集
        
        1. `Python` : `numpy:ndarray`
        2. `C++`    : `tensorflow::Tensor`
        
    * 变量维护状态
    * `feed` / `fetch`(`session.run`) 获取数据
    
2. 基本解释
    
    1. 计算图
    
        * 构建图 : 
        
            1. op 的执行步骤被描述成一个图
            
            2. 一般都是在构建阶段设计构建表示神经网络
            
            3. 构建方式
                
                1. 源`Op` : 输入的`Op`节点,一般都是`tf.placeholder`, `tf.constant`等等
                2. `Op` : `op` 构造器的返回值代表被构造出的 `op` 的输出, 这些返回值可以传递给其它 `op` 构造器作为输入
                
            4. 默认图构建，我们一般来说，使用默认图已经足够了
            
        * 执行图 : 
            
            1. 使用会话执行执行图中的 op.
            
            2. 执行的过程中，反复计算节点和训练神经网络
            
            3. 返回`numpy:ndarray`对象
            
    2. Tensor
    
    3. Variable : 
    
        1. 变量维护图执行过程中的状态信息
        2. 在神经网络中，我们通常可以将网络参数作为权值保存在变量中并在训练过程中不断的更新

In [2]:
# 创建一个常量 op, 产生一个 1x2 矩阵. 这个 op 被作为一个节点
# 加到默认图中.

# 构造器的返回值代表该常量 op 的返回值.
matrix1 = tf.constant([[3., 3.]])

# 创建另外一个常量 op, 产生一个 2x1 矩阵.
matrix2 = tf.constant([[2.],[2.]])

# 创建一个矩阵乘法 matmul op , 把 'matrix1' 和 'matrix2' 作为输入. 结果是 (1, 1)的矩阵
# 返回值 'product' 代表矩阵乘法的结果.
product = tf.matmul(matrix1, matrix2)

In [3]:
# 启动图
with tf.Session() as sess:
    result = sess.run(product)
    print(result, type(result))

[[ 12.]] <class 'numpy.ndarray'>


In [6]:
# 让变量成为一个计数器
# 创建一个变量, 初始化为标量 0.
state = tf.Variable(0, name="counter")

one = tf.constant(1)
new_value = tf.add(state, one)
# 更新variable的状态
update = tf.assign(state, new_value)

# 启动图后, 变量必须先经过`初始化` (init) op 初始化,
# 首先必须增加一个`初始化` op 到图中.
init_op = tf.global_variables_initializer()

# 启动图, 运行 op
with tf.Session() as sess:
    # 运行 'init' op
    sess.run(init_op)
    # 打印 'state' 的初始值
    print(sess.run(state))
    # 运行 op, 更新 'state', 并打印 'state'
    for _ in range(3):
        sess.run(update)
        print(sess.run(state))

0
1
2
3
